In [ ]:
%load_ext micropython_magic
%reload_ext micropython_magic

In [ ]:
!mpremote cp -r ../../mp_libs/power :

In [ ]:
# %%micropython

import time
from machine import Pin
from mp_libs.power import powerfeather
from mp_libs.power import bq2562x as bq
from mp_libs.power import lc709204f as fg

pf = powerfeather.PowerFeather(batt_type=powerfeather.BatteryType.GENERIC_3V7, batt_cap=1050)

def cb_button(pin: Pin) -> None:
    print("Button Pressed! Toggling charging.")
    if pf._charger.charging_enable:
        pf.batt_charging_enable(False)
    else:
        pf.batt_charging_enable(True)

pf.register_button_irq(cb_button)
term_current = pf._charger.term_current
supply_voltage = 0
supply_current = 0
batt_voltage = 0
batt_current = 0
batt_charge = 0
batt_cycles = 0
batt_health = 0
batt_time_left = 0
status = None

while True:
    pf.led_toggle()

    # These commands use the batt charger IC and therefore don't require a batt connected to work.
    supply_voltage = pf.supply_voltage()
    supply_current = pf.supply_current()
    batt_current = pf.batt_current()
    status = pf.batt_charging_status()

    # These commands use the fuel gauge IC and therefore require a battery to be connected in order to work.
    try:
        batt_voltage = pf.batt_voltage()
        batt_charge = pf.batt_charge()
        batt_cycles = pf.batt_cycles()
        batt_health = pf.batt_health()
        batt_time_left = pf.batt_time_left()
    except (powerfeather.BatteryError, fg.FuelGaugeError):
        pass

    print(f"[Supply]  Voltage: {supply_voltage:6} mV\tCurrent: {supply_current:6} mA\tPower: {supply_voltage * supply_current / 1_000_000.0:6.2f} W")
    print(f"[Battery] Voltage: {batt_voltage:6} mv\tCurrent: {batt_current:6} mA\tPower: {batt_voltage * batt_current / 1_000_000.0:6.2f} W")
    print(f"[Battery] Charge:  {batt_charge:6}%   \tCycles:  {batt_cycles:6}   \tHealth: {batt_health:6}%   \tStatus: {status:6}\tTerm: {term_current:6} mA")
    print(f"Time Left: {batt_time_left} min")

    time.sleep_ms(300)
